In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

In [5]:
def parse_ria_news(date_from=None, max_news=5):
    """
    Парсит новости с РИА Новости.
    
    Args:
        date_from (str, optional): Дата в формате 'YYYY-MM-DD'. Парсинг новостей начиная с этой даты.
        max_news (int, optional): Максимальное количество новостей для парсинга.
    
    Returns:
        pd.DataFrame: DataFrame с колонками ['date', 'title', 'text', 'link'].
    """
    
    base_url = "https://ria.ru"
    section_url = "https://ria.ru/world/"
    news_list = []

    response = requests.get(section_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
    else:
        print("Ошибка чето")
    
    news_cards = soup.find_all("div", class_="list-item__content")
    
    for card in news_cards:

        title_tag = card.find("a", class_="list-item__title")
        if not title_tag:
            continue
        title = title_tag.text.strip()
        link = title_tag['href']

        date_tag = card.find("span", class_="list-item__date")
        if date_tag:
            date_text = date_tag.text.strip()
            try:
                news_date = datetime.strptime(date_text, "%d.%m.%Y %H:%M")
            except:
                news_date = None
        else:
            news_date = None

        if date_from:
            date_from_dt = datetime.strptime(date_from, "%Y-%m-%d")
            if news_date and news_date < date_from_dt:
                continue

        news_response = requests.get(link)
        news_soup = BeautifulSoup(news_response.text, "html.parser")

        date_match = re.search(r'/(\d{8})/', link)
        if date_match:
            news_date = datetime.strptime(date_match.group(1), "%Y%m%d")
        else:
            news_date = None

        text_tag = news_soup.find("div", class_="article__body")
        text = text_tag.get_text(separator="\n").strip() if text_tag else ""

        news_list.append({
            "date": news_date,
            "title": title,
            "text": text,
            "link": link
        })

        if max_news and len(news_list) >= max_news:
            break

    df = pd.DataFrame(news_list)
    return df


In [4]:
df_news = parse_ria_news(date_from="2025-10-01", max_news=5)
df_news

,date,title,text,link
0,2025-11-07,"СМИ узнали, кого Рубио назвал главным кандидат...","МОСКВА, 7 ноя - РИА Новости.\n Госсекретарь СШ...",https://ria.ru/20251107/ssha-2053443116.html
1,2025-11-07,Британия сняла санкции с главы Сирии аш-Шараа,"ЛОНДОН, 7 ноя - РИА Новости.\n Великобритания ...",https://ria.ru/20251107/britanija-2053442788.html
2,2025-11-07,"В правительстве Венгрии сообщили, о чем Оран б...","МОСКВА, 7 ноя - РИА Новости.\n Премьер-министр...",https://ria.ru/20251107/orban-2053442183.html
3,2025-11-07,В НАТО признали нехватку систем ПВО в Европе,"КИШИНЕВ, 7 ноя - РИА Новости.\n Генеральный се...",https://ria.ru/20251107/pvo-2053441743.html
4,2025-11-07,Россия осудила осквернение Эстонией останков б...,"МОСКВА, 7 ноя - РИА Новости.\n Россия осуждает...",https://ria.ru/20251107/rossija-2053440461.html


In [15]:
print(df_news.loc[0].title, df_news.loc[0].text, df_news.loc[0].link, sep="\n")

США опровергли заявления о поставках Пакистану ракет средней дальности
НЬЮ-ДЕЛИ, 10 окт – РИА Новости.
 Посольство США в Индии опровергло заявления о якобы планируемых поставках Пакистану новых усовершенствованных ракет класса "воздух-воздух" средней дальности (AMRAAM, Advanced Medium-Range Air-to-Air Missile). 
"Тридцатого сентября 2025 года министерство обороны США опубликовало список стандартных объявлений о контрактах, в которых упоминалось изменение существующего контракта на поставку вооружений и запасных частей для ряда стран, включая 
Пакистан
. Администрация хотела бы подчеркнуть, что, вопреки ложным сообщениям СМИ, упомянутое изменение контракта не касается поставок Пакистану новых усовершенствованных ракет класса "воздух-воздух" средней дальности (AMRAAM)", - сообщило посольство. 
На Западе назвали 15 стран, способных нанести ядерный удар
18 сентября, 17:22
Там отметили, что поставка вооружений не предусматривает модернизации каких-либо существующих возможностей Пакистана. 
